In [2]:
import os
from rich import print as rprint
import pymupdf
import pymupdf4llm
import re
from statistics import median
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from itertools import groupby
from operator import itemgetter


def markdown_format(page_lines, page_num):
    '''
    Convert page lines to markdown and insert metadata.
    '''
    md_lines = []
    for line in page_lines:
        text = line["text"]
        block_num = line["block"]
        is_header = line["is_header"]

        # Instead of splitting, just wrap the whole text as Document
        doc = Document(
            page_content=text,
            metadata={
                "page_number": page_num,
                "block_number": block_num,
                "is_header": is_header
            }
        )
        md_lines.append(doc)
    return md_lines


recur_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    length_function=len,
    is_separator_regex=False,
)

def recursive_chunk_splitter(md_lines):
    '''
    Convert page lines to markdown.
    '''
    chunks_with_metadata = [] # use this to accumlate chunk records

    for item in md_lines: # pull these keys from the json
        item_text = item.page_content
        item_metadata = item.metadata
        item_text_chunks = recur_text_splitter.split_text(item_text) # split the text into chunks
        chunk_seq_id = 0

        for chunk in item_text_chunks:# only take the first 20 chunks
            item_metadata['chunk_seq_id'] = chunk_seq_id
            item_metadata['block_id'] = f'{item.metadata["pdf_file_name"]}-{item.metadata["page_number"]}-{item.metadata["block_number"]}-chunk{chunk_seq_id:04d}'
            copy_metadata = item_metadata.copy()
            chunks_with_metadata.append(Document(page_content=chunk, metadata=copy_metadata))
            chunk_seq_id += 1

    return chunks_with_metadata


def insert_additional_metadata(md, page, file_path, page_nums):
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    
    title = page['metadata']['title'] if page['metadata']['title'] else file_name
    page_count = page['metadata']['page_count']
    toc_items = page.get('toc_items', [])

    # Handle TOC gracefully
    if toc_items and toc_items[0].strip() != "":
        toc_item = toc_items[0]
        chapter_name = toc_item[1] if toc_item[1].strip() != "" else "__Unknown__"
        chapter_number = toc_item[0] if toc_item[0].strip() != "" else "__Unknown__"
    else:
        chapter_name = "__Unknown__"
        chapter_number = "__Unknown__"

    # Default source is page number
    source = page['metadata']['page']

    for line in md:
        initial_meta = line.metadata.copy()

        # Update source if header
        if initial_meta.get("is_header") is True:
            source = line.page_content  # Or whatever unique identifier you want

        # Rebuild metadata
        line.metadata = {}
        line.metadata['source'] = source
        line.metadata['is_header'] = initial_meta.get("is_header")
        line.metadata['pdf_file_name'] = file_name
        line.metadata['toc'] = toc_items
        line.metadata['block_number'] = initial_meta.get("block_number", -1)
        line.metadata['page_number'] = page['metadata']['page']
        line.metadata['has_images'] = False
        line.metadata['alt_title'] = title
        line.metadata['page_count'] = page_count
        line.metadata['chapter_name'] = chapter_name
        line.metadata['chapter_number'] = chapter_number
        line.metadata['page_id'] = f'{file_name}-{page["metadata"]["page"]}'

    return md


def span_is_header(span, size_median):
    size = span["size"]
    flags = span["flags"]
    text  = span["text"].strip()
    font  = span["font"].lower()

    # Header detection logic
    is_header = False
    if size > 1.5 * size_median:
        is_header = True
    if size >= 18:
        is_header = True
    if flags == 20:
        is_header = True

    line_obj = {
        "is_header": is_header,
        "line": span["line"],
        "block": span["block"],
        "text": span["text"]
    }

    return line_obj
    

def make_header_detector():
    returns_list = []

    def my_header_detector(span, page=None):
        ret_val = ""
        returns_list.append(span)
        return ret_val
        
    return my_header_detector, returns_list

    

In [16]:
def process_pdf(pdf_file, file_path, page_nums=None):
    '''
    Process a single page of a PDF file.
    '''

    book_array = []
    prev_header = {}
    has_images = False

    doc = pymupdf.open(file_path)

    # get Table of content
    # rprint(doc.get_toc())

    for num in range(len(pdf_file)):
        detector_func, page_spans = make_header_detector()

        # pymupdf4llm extracts content from pdfs and allows us to access font size, block position and line position of text in the pdf
        md_output = pymupdf4llm.to_markdown(
            file_path,
            page_chunks=True,
            pages = [num],
            extract_words=True,
            hdr_info=detector_func
        )
        
        # intialize structures to hold words/lines
        text_array = []

        # Calculate median size
        sizes = [span["size"] for span in page_spans if span["size"] > 0]
        size_median = median(sizes) if sizes else 9

        # Assess if line is a header and build text_array of objs
        text_array = [span_is_header(span, size_median) for span in page_spans]

        # sort by block and line
        sorted_lines = sorted(text_array, key=lambda x: (x["block"], x["line"]))


        # merge lines into their respective blocks
        merged_blocks = []
        for block_num, group in groupby(sorted_lines, key=lambda x: x["block"]):
            block_lines = list(group)
            merged_text = " ".join(line["text"] for line in block_lines)
            is_header = any(line["is_header"] for line in block_lines)

            merged_blocks.append({
                "block": block_num,
                "is_header": is_header,
                "text": merged_text
            })

        intitial_md = markdown_format(merged_blocks, num)
        
        final_md = insert_additional_metadata(intitial_md, md_output[0], file_path, num)
            
        # Split markdown into chunks
        md_lines_chunked = recursive_chunk_splitter(final_md)

        book_array.extend(md_lines_chunked)

    return book_array


complete_path = "pdf_files/NewYorkCityFor Dummies.pdf"

pdf_file = pymupdf.open(complete_path, filetype="pdf")
page_nums = None

llm_ready_doc = process_pdf(pdf_file, complete_path, page_nums)
rprint(llm_ready_doc)


[
    [1, 'New York City For Dummies 7th Edition', 1],
    [2, 'About the Author', 5],
    [2, 'Table of Contents', 11],
    [2, 'Introduction', 19],
    [3, 'About This Book', 19],
    [3, 'Conventions Used in This Book', 20],
    [3, 'How This Book Is Organized', 20],
    [3, 'Icons Used in This Book', 22],
    [3, 'Where to Go from Here', 22],
    [2, 'Part I: Introducing New York City', 23],
    [3, 'Chapter 1: Discovering the Best of New York City', 25],
    [4, 'Best Events', 25],
    [4, 'Best Hotels', 32],
    [4, 'Best Restaurants', 33],
    [4, 'Best Attractions', 34],
    [4, 'Best Shopping', 36],
    [4, 'Best Culture and Nightlife', 36],
    [3, 'Chapter 2: Digging Deeper into New York City', 38],
    [4, 'Hunting Down a Little History', 38],
    [4, 'Looking at Local Architecture', 41],
    [4, 'Recommended Books, Movies, and Podcasts', 42],
    [3, 'Chapter 3: Deciding When to Go', 45],
    [4, 'Revealing the Secrets of the Seasons', 45],
    [4, 'Marking Your Calendar: Year-Round New York', 46],
    [2, 'Part II: Planning Your Trip to New York City', 51],
    [3, 'Chapter 4: Managing Your Money', 53],
    [4, 'Planning Your Budget', 53],
    [4, 'Cutting Costs, but Not the Fun', 57],
    [4, 'Handling Money', 58],
    [3, 'Chapter 5: Getting to New York City', 61],
    [4, 'Choosing the Airport', 61],
    [4, 'Flying to New York', 62],
    [4, 'Driving to New York City', 64],
    [4, 'Arriving by Other Means', 65],
    [4, 'Joining an Escorted Tour', 66],
    [4, 'Choosing a Package Tour', 67],
    [3, 'Chapter 6: Catering to Special Travel Needs or Interests', 68],
    [4, 'Traveling with the Brood: Advice for Families', 68],
    [4, 'Making Age Work for You: Advice for Seniors', 70],
    [4, 'Accessing New York City: Advice for Travelers with Disabilities', 70],
    [4, 'Following the Rainbow: Advice for LGBT Travelers', 72],
    [4, 'Tying the Knot: Advice for Travelers in Love', 73],
    [3, 'Chapter 7: Taking Care of the Remaining Details', 75],
    [4, 'Renting a Car: Not in New York!', 75],
    [4, 'Playing It Safe: Travel and Medical Insurance', 76],
    [4, 'Staying Healthy When You Travel', 77],
    [4, 'Staying Connected by Cellphone or E-Mail', 78],
    [4, 'Keeping Up with Airline Security Measures', 80],
    [2, 'Part III: Settling into New York City', 81],
    [3, 'Chapter 8: Arriving and Getting Oriented', 83],
    [4, 'Getting from the Airport to Your Hotel', 83],
    [4, 'Arriving by Train', 89],
    [4, 'Arriving by Car', 90],
    [4, 'Figuring Out the Neighborhoods', 91],
    [4, 'Finding Information after You Arrive', 96],
    [4, 'Getting around New York', 97],
    [4, 'Seeing New York on Foot', 103],
    [3, 'Chapter 9: Checking In at New York City’s Best Hotels', 105],
    [4, 'Getting to Know Your Options', 105],
    [4, 'Finding the Best Room at the Best Rate', 106],
    [4, 'Arriving without a Reservation (Not Recommended)', 108],
    [4, 'New York City’s Best Hotels', 109],
    [4, 'Runner-Up Hotels', 132],
    [4, 'Index of Accommodations by Neighborhood', 134],
    [4, 'Index of Accommodations by Price', 135],
    [3, 'Chapter 10: Dining and Snacking in New York City', 136],
    [4, 'Getting the Dish on the Local Scene', 136],
    [4, 'New York’s Best Restaurants', 139],
    [4, 'Dining and Snacking on the Go', 168],
    [4, 'Index of Establishments by Neighborhood', 177],
    [4, 'Index of Establishments by Cuisine', 178],
    [4, 'Index of Establishments by Price', 180],
    [2, 'Part IV: Exploring New York City', 183],
    [3, 'Chapter 11: New York City’s Top Sights', 185],
    [4, 'New York City’s Top Sights', 192],
    [4, 'Finding More Cool Things to See and Do', 206],
    [4, 'Surveying the New York Sports Scene', 220],
    [4, 'Seeing New York by Guided Tour', 222],
    [3, 'Chapter 12: Shopping in New York City', 227],
    [4, 'Surveying the Shopping Scene', 227],
    [4, 'Knowing the Big Names', 234],
    [4, 'Shopping in Open-Air Markets', 236],
    [4, 'Discovering the 

False

[]